In [2]:
import dask.distributed
import dask.utils
import numpy as np
import planetary_computer as pc
import xarray as xr
from IPython.display import display
from pystac_client import Client
import geopandas as gpd
import requests
from shapely.geometry import box
from odc.geo.geobox import GeoBox
from odc.stac import configure_rio, stac_load
import pickle
%matplotlib inline

In [11]:
client = dask.distributed.Client()
configure_rio(cloud_defaults=True, client=client)
display(client)

/home/laserglaciers/miniconda3/envs/py310v2/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46585 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46585/status,
Dashboard: http://127.0.0.1:46585/status,Workers: 4
Total threads: 12,Total memory: 15.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44109,Workers: 4
Dashboard: http://127.0.0.1:46585/status,Total threads: 12
Started: Just now,Total memory: 15.45 GiB
Comm: tcp://127.0.0.1:34545,Total threads: 3
Dashboard: http://127.0.0.1:47331/status,Memory: 3.86 GiB
Nanny: tcp://127.0.0.1:45099,


In [12]:
item_pkls = './item_pkls/kanger_image_item_list.pkl'

with open(item_pkls, 'rb') as f:
    kanger_stack_list = pickle.load(f)
    

kanger_stack_list

[<Item id=S2A_MSIL2A_20160713T141952_R096_T25WES_20210212T041530>,
 <Item id=S2A_MSIL2A_20190304T140051_R010_T25WES_20201030T131654>,
 <Item id=S2B_MSIL2A_20200912T140739_R053_T25WES_20200918T131859>,
 <Item id=S2B_MSIL2A_20240514T140729_R053_T25WES_20240514T163024>]

In [18]:
grid_path = '/media/laserglaciers/upernavik/iceberg_py/geoms/kanger/kanger_box_utm25N_v2.gpkg'
grid = gpd.read_file(grid_path)

In [19]:
resolution = 10
SHRINK = 1
if client.cluster.workers[0].memory_manager.memory_limit < dask.utils.parse_bytes("4G"):
    SHRINK = 8  # running on Binder with 2Gb RAM

if SHRINK > 1:
    resolution = resolution * SHRINK


image_stac = stac_load(
    kanger_stack_list,
    bands=["red", "green", "blue"],
    resolution=resolution,
    chunks={"x": 2048, "y": 2048},
    patch_url=pc.sign,
    # force dtype and nodata
    dtype="uint16",
    nodata=0,
    geopolygon=grid
)

In [20]:
image_stac

<xarray.Dataset> Size: 302MB
Dimensions:      (y: 3145, x: 4002, time: 4)
Coordinates:
  * y            (y) float64 25kB 7.617e+06 7.617e+06 ... 7.586e+06 7.586e+06
  * x            (x) float64 32kB 4.954e+05 4.954e+05 ... 5.354e+05 5.354e+05
    spatial_ref  int32 4B 32625
  * time         (time) datetime64[ns] 32B 2016-07-13T14:19:52.026000 ... 202...
Data variables:
    red          (time, y, x) uint16 101MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    green        (time, y, x) uint16 101MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>
    blue         (time, y, x) uint16 101MB dask.array<chunksize=(1, 2048, 2048), meta=np.ndarray>

In [21]:
grid.crs

<Projected CRS: EPSG:32625>
Name: WGS 84 / UTM zone 25N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 36°W and 30°W, northern hemisphere between equator and 84°N, onshore and offshore. Greenland.
- bounds: (-36.0, 0.0, -30.0, 84.0)
Coordinate Operation:
- name: UTM zone 25N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich